In [1]:
import requests
import json
import pandas as pd
import numpy as np
import datetime
import calendar
from config import bls_api_key


## Construction spending

In [2]:
start_year = "2006"
end_year = "2015"

In [3]:
# Pull a csv
cs_df = pd.read_csv("Resources/totsatime.csv")

# center on value filled cells

cs_df.columns = cs_df.iloc[2,:]
cs_df = cs_df.loc[3:329,:]

# Rename columns

cs_df = cs_df.rename(columns={"Total\n\rConstruction1": "Total Construction",
                             "Total\n\rResidential": "Total Residential",
                             "Total\n\rNonresidential": "Total Nonresidential",
                             "Total\n\rLodging": "Total Lodging",
                             "Total\n\rOffice": "Total Office",
                             "Total\n\rCommercial": "Total Commercial",
                             "Total\n\rHealth care": "Total Health Care",
                             "Total\n\rEducational": "Total Educational",
                             "Total\n\rReligious": "Total Religious",
                             "Total\n\rPublic safety": "Total Public safety",
                             "Total\n\rAmusement and recreation": "Total Amusement and recreation",
                             "Total\n\rTransportation": "Total Transportation",
                             "Total\n\rCommunication": "Total Communication",
                             "Total\n\rPower": "Total Power",
                             "Total\n\rHighway and street": "Total Highway and street",
                             "Total\n\rSewage and waste disposal": "Total Sewage and waste disposal",
                             "Total\n\rWater supply": "Total Water supply",
                             "Total\n\rConservation and development": "Total Conservation and development",
                             "Total\n\rManufacturing": "Total Manufacturing",
                             "Total\n\rPrivate Construction2": "Total Private Construction",
                             "Private\n\rResidential": "Private Residential",
                             "Private\n\rNonresidential": "Private Nonresidential",
                             "Private\n\rLodging": "Private Lodging",
                             "Private\n\rOffice": "Private Office",
                             "Private\n\rCommercial": "Private Commercial",
                             "Private\n\rHealth care": "Private Health care",
                             "Private\n\rEducational": "Private Educational",
                             "Private\n\rReligious": "Private Religious",
                             "Private\n\rAmusement and recreation": "Private Amusement and recreation",
                             "Private\n\rTransportation": "Private Transportation",
                             "Private\n\rCommunication": "Private Communication",
                             "Private\n\rPower": "Private Power",
                             "Private\n\rManufacturing": "Private Manufacturing",
                             "Total\n\rPublic Construction3": "Total Public Construction",
                             "Public\n\rResidential": "Public Residential",
                             "Public\n\rNonresidential": "Public Nonresidential",
                             "Public\n\rOffice": "Public Office",
                             "Public\n\rCommercial": "Public Commercial",
                             "Public\n\rHealth care": "Public Health care",
                             "Public\n\rEducation": "Public Education",
                             "Public\n\rPublic safety": "Public Public safety",
                             "Public\n\rAmusement and recreation": "Public Amusement and recreation",
                             "Public\n\rTransportation": "Public Transportation",
                             "Public\n\rPower": "Public Power",
                             "Public\n\rHighway and street": "Public Highway and street",
                             "Public\n\rSewage and waste disposal": "Public Sewage and waste disposal",
                             "Public\n\rWater supply": "Public Water supply",
                             "Public\n\rConservation and development": "Public Conservation and development",
                             })

# Remove duplicate date columns

cs_df = cs_df.loc[:,~cs_df.columns.duplicated()]

# Center on year range and handle exception of end year 2020

cs_df.reset_index(drop=True, inplace=True)

cs_df.iloc[0:3,0] = cs_df.iloc[0:3,0].str[:-1]

if end_year == "2020":
    cs_df = cs_df.iloc[cs_df.index[cs_df["Date"] == f"Feb-20"][0]: (cs_df.index[cs_df["Date"] == f"Jan-{start_year[2:]}"][0])+1,:]
else:
    cs_df = cs_df.iloc[cs_df.index[cs_df["Date"] == f"Dec-{end_year[2:]}"][0]: (cs_df.index[cs_df["Date"] == f"Jan-{start_year[2:]}"][0])+1,:]

# Reformat date column

cs_df.replace(to_replace=["Jan","Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
              value=["01","02","03", "04", "05", "06", "07", "08", "09", "10", "11", "12"],
              regex=True,
              inplace=True)

cs_df["Date"] = "20" + cs_df["Date"].str[-2:] + "-" + cs_df["Date"].str[:2]

# Sort data to ascending

cs_df.sort_values(by="Date", ascending=True, inplace=True)

cs_df.reset_index(drop=True, inplace=True)

# turn data in floats

cs_df = cs_df.replace(",","", regex=True)

for column in cs_df.columns[1:]:
    cs_df[f"{column}"] = cs_df[f"{column}"].astype(float)

cs_df.head()

2,Date,Total Construction,Total Residential,Total Nonresidential,Total Lodging,Total Office,Total Commercial,Total Health Care,Total Educational,Total Religious,...,Public Health care,Public Education,Public Public safety,Public Amusement and recreation,Public Transportation,Public Power,Public Highway and street,Public Sewage and waste disposal,Public Water supply,Public Conservation and development
0,2006-01,1195698.0,681319.0,514379.0,13561.0,46915.0,76900.0,35678.0,83383.0,7435.0,...,5879.0,69946.0,6980.0,8433.0,17716.0,8156.0,66507.0,22299.0,14377.0,4543.0
1,2006-02,1205060.0,683903.0,521157.0,14272.0,49018.0,74345.0,36434.0,83987.0,7839.0,...,6192.0,70471.0,7327.0,8634.0,17774.0,9358.0,68286.0,22826.0,13911.0,4445.0
2,2006-03,1205018.0,674306.0,530712.0,15619.0,49179.0,74922.0,36751.0,84429.0,8032.0,...,6202.0,70861.0,7217.0,8869.0,18843.0,8778.0,70141.0,23328.0,14614.0,5058.0
3,2006-04,1192849.0,656452.0,536398.0,17444.0,52392.0,73856.0,37058.0,83236.0,7817.0,...,6290.0,69521.0,7336.0,8953.0,19045.0,9444.0,73543.0,22427.0,14276.0,5206.0
4,2006-05,1180699.0,640166.0,540532.0,17269.0,53233.0,75490.0,37688.0,83137.0,7911.0,...,6349.0,69345.0,7322.0,9440.0,18988.0,8830.0,73387.0,21858.0,14903.0,5269.0


## Demographic unemployment rate

In [4]:
# Pull in demographic data from Bureau of Labor Statistics

url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers = {"content-type": "application/json"}

target_series = ["LNS14000000", "LNS14000001", "LNS14000002", "LNS14000003", "LNS14000004", "LNS14000005",
                 "LNS14000006", "LNS14000007", "LNS14000008", "LNS14032183", "LNS14000009", "LNS14000012",
                 "LNS14000013", "LNS14000014", "LNS14000015", "LNS14000016", "LNS14000017", "LNS14000018",
                 "LNS14000019", "LNS14000020", "LNS14000021", "LNS14000022", "LNS14000023", "LNS14000036",
                 "LNS14000037", "LNS14000038", "LNS14000089", "LNS14000164", "LNS14000327", "LNS14000091",
                 "LNS14000173", "LNS14000334", "LNS14000093", "LNS14000182", "LNS14000341", "LNS14024230",
                 "LNS14024231", "LNS14024232", "LNS14000048", "LNS14000049", "LNS14000050", "LNS14027659",
                 "LNS14027660", "LNS14027689", "LNS14027662", "LNS14000024", "LNS14000025", "LNS14000026",
                 "LNS14000027", "LNS14000028"]


parameters = json.dumps({"seriesid": target_series,
             "startyear": start_year,
             "endyear": end_year,
             "registrationkey": bls_api_key})

p = requests.post(url, data=parameters, headers=headers)

json_data5 = json.loads(p.text)

In [5]:
# Pull in demographic data from Bureau of Labor Statistics

url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers = {"content-type": "application/json"}

target_series = ["LNS14000029", "LNS14000030", "LNS14000031", "LNS14000032",
                 "LNS14000033", "LNS14000034", "LNS14000035"]


parameters = json.dumps({"seriesid": target_series,
             "startyear": start_year,
             "endyear": end_year,
             "registrationkey": bls_api_key})

p = requests.post(url, data=parameters, headers=headers)

json_data6 = json.loads(p.text)

In [6]:
Year = []
month = []
date = []

for data_point in np.arange(len(json_data5["Results"]["series"][0]["data"])):
        Year.append(f"{json_data5['Results']['series'][0]['data'][data_point]['year']}")
        month.append(f"{json_data5['Results']['series'][0]['data'][data_point]['periodName']}")
        date.append(f"{json_data5['Results']['series'][0]['data'][data_point]['year']}-{json_data5['Results']['series'][0]['data'][data_point]['period'][1:]}")

In [7]:
# Create DataFrame

unemployment_df = pd.DataFrame({"Date": date, "Year": Year, "month": month,
                                "LNS14000000": "", "LNS14000001": "", "LNS14000002": "",
                                "LNS14000003": "", "LNS14000004": "", "LNS14000005": "",
                                "LNS14000006": "", "LNS14000007": "", "LNS14000008": "",
                                "LNS14032183": "", "LNS14000009": "", "LNS14000012": "",
                                "LNS14000013": "", "LNS14000014": "", "LNS14000015": "",
                                "LNS14000016": "", "LNS14000017": "", "LNS14000018": "",
                                "LNS14000019": "", "LNS14000020": "", "LNS14000021": "",
                                "LNS14000022": "", "LNS14000023": "", "LNS14000024": "",
                                "LNS14000025": "", "LNS14000026": "", "LNS14000027": "",
                                "LNS14000028": "", "LNS14000029": "", "LNS14000030": "",
                                "LNS14000031": "", "LNS14000032": "", "LNS14000033": "",
                                "LNS14000034": "", "LNS14000035": "", "LNS14000036": "",
                                "LNS14000037": "", "LNS14000038": "", "LNS14000089": "",
                                "LNS14000164": "", "LNS14000327": "", "LNS14000091": "",
                                "LNS14000173": "", "LNS14000334": "", "LNS14000093": "",
                                "LNS14000182": "", "LNS14000341": "", "LNS14024230": "",
                                "LNS14024231": "", "LNS14024232": "", "LNS14000048": "",
                                "LNS14000049": "", "LNS14000050": "", "LNS14027659": "",
                                "LNS14027660": "", "LNS14027689": "", "LNS14027662": ""})

unemployment_df.head()

,Date,Year,month,LNS14000000,LNS14000001,LNS14000002,LNS14000003,LNS14000004,LNS14000005,LNS14000006,...,LNS14024230,LNS14024231,LNS14024232,LNS14000048,LNS14000049,LNS14000050,LNS14027659,LNS14027660,LNS14027689,LNS14027662
0,2015-12,2015,December,,,,,,,,...,,,,,,,,,,
1,2015-11,2015,November,,,,,,,,...,,,,,,,,,,
2,2015-10,2015,October,,,,,,,,...,,,,,,,,,,
3,2015-09,2015,September,,,,,,,,...,,,,,,,,,,
4,2015-08,2015,August,,,,,,,,...,,,,,,,,,,


In [8]:
# Fills values

for series5 in np.arange(len(json_data5["Results"]["series"])):
    for data_point5 in np.arange(len(json_data5["Results"]["series"][series5]["data"])):
        unemployment_df.loc[data_point5, json_data5["Results"]["series"][series5]["seriesID"]] = json_data5["Results"]["series"][series5]["data"][data_point5]["value"]

for series in np.arange(len(json_data6["Results"]["series"])):
    for data_point in np.arange(len(json_data6["Results"]["series"][series]["data"])):
        unemployment_df.loc[data_point, json_data6["Results"]["series"][series]["seriesID"]] = json_data6["Results"]["series"][series]["data"][data_point]["value"]

In [9]:
# Sets data to float

for column in unemployment_df.columns[3:]:
    unemployment_df[f"{column}"] = unemployment_df[f"{column}"].astype(float)

# Rename column labels

unemployment_df = unemployment_df.rename(columns={"LNS14000000": "Unemployment Rate, 16yo and over",
                                                  "LNS14000001": "Unemployment Rate, Men, 16yo and over",
                                                  "LNS14000002": "Unemployment Rate, Women, 16yo and over",
                                                  "LNS14000003": "Unemployment Rate, White, 16yo and over",
                                                  "LNS14000004": "Unemployment Rate, White, Men, 16yo and over",
                                                  "LNS14000005": "Unemployment Rate, White, Women, 16yo and over",
                                                  "LNS14000006": "Unemployment Rate, African American, 16yo and over",
                                                  "LNS14000007": "Unemployment Rate, African American, Men, 16yo and over",
                                                  "LNS14000008": "Unemployment Rate, African American, Women, 16yo and over",
                                                  "LNS14032183": "Unemployment Rate, Asian, 16yo and over",
                                                  "LNS14000009": "Unemployment Rate, Hispanic or Latino, 16yo and over",
                                                  "LNS14000012": "Unemployment Rate, 16-19yo",
                                                  "LNS14000013": "Unemployment Rate, Men, 16-19yo",
                                                  "LNS14000014": "Unemployment Rate, Women, 16-19yo",
                                                  "LNS14000015": "Unemployment Rate, White, 16-19yo",
                                                  "LNS14000016": "Unemployment Rate, White, Men, 16-19yo",
                                                  "LNS14000017": "Unemployment Rate, White, Women, 16-19yo",
                                                  "LNS14000018": "Unemployment Rate, African American, 16-19yo",
                                                  "LNS14000019": "Unemployment Rate, African American, Men, 16-19yo",
                                                  "LNS14000020": "Unemployment Rate, African American, Women, 16-19yo",
                                                  "LNS14000021": "Unemployment Rate, Hispanic or Latino, 16-19yo",
                                                  "LNS14000022": "Unemployment Rate, Hispanic or Latino, Men, 16-19yo",
                                                  "LNS14000023": "Unemployment Rate, Hispanic or Latino, Women, 16-19yo",
                                                  "LNS14000024": "Unemployment Rate, 20yo and over",
                                                  "LNS14000025": "Unemployment Rate, Men, 20yo and over",
                                                  "LNS14000026": "Unemployment Rate, Women, 20yo and over",
                                                  "LNS14000027": "Unemployment Rate, White, 20yo and over",
                                                  "LNS14000028": "Unemployment Rate, White, Men, 20yo and over",
                                                  "LNS14000029": "Unemployment Rate, White, Women, 20yo and over",
                                                  "LNS14000030": "Unemployment Rate, African American, 20yo and over",
                                                  "LNS14000031": "Unemployment Rate, African American, Men, 20yo and over",
                                                  "LNS14000032": "Unemployment Rate, African American, Women, 20yo and over",
                                                  "LNS14000033": "Unemployment Rate, Hispanic or Latino",
                                                  "LNS14000034": "Unemployment Rate, Hispanic or Latino, Men, 20yo and over",
                                                  "LNS14000035": "Unemployment Rate, Hispanic or Latino, Women, 20yo and over",
                                                  "LNS14000036": "Unemployment Rate, 20-24yo",
                                                  "LNS14000037": "Unemployment Rate, Men, 20-24yo",
                                                  "LNS14000038": "Unemployment Rate, Women, 20-24yo",
                                                  "LNS14000089": "Unemployment Rate, 25-34yo",
                                                  "LNS14000164": "Unemployment Rate, Men, 25-34yo",
                                                  "LNS14000327": "Unemployment Rate, Women, 25-34yo",
                                                  "LNS14000091": "Unemployment Rate, 35-44yo",
                                                  "LNS14000173": "Unemployment Rate, Men, 35-44yo",
                                                  "LNS14000334": "Unemployment Rate, Women, 35-44yo",
                                                  "LNS14000093": "Unemployment Rate, 45-54yo",
                                                  "LNS14000182": "Unemployment Rate, Men, 45-54yo",
                                                  "LNS14000341": "Unemployment Rate, Women, 45-54yo",
                                                  "LNS14024230": "Unemployment Rate, 55yo and over",
                                                  "LNS14024231": "Unemployment Rate, Men, 55yo and over",
                                                  "LNS14024232": "Unemployment Rate, Women, 55yo and over",
                                                  "LNS14000048": "Unemployment Rate, 25yo and over",
                                                  "LNS14000049": "Unemployment Rate, Men, 25yo and over",
                                                  "LNS14000050": "Unemployment Rate, Women, 25yo and over",
                                                  "LNS14027659": "Unemployment Rate, Less than High School diploma, 25yo and over",
                                                  "LNS14027660": "Unemployment Rate, High school-no college, 25yo and over",
                                                  "LNS14027689": "Unemployment Rate, Some college, 25yo and over",
                                                  "LNS14027662": "Unemployment Rate, Bachelor's degree or higher, 25yo and over"})

# Sort data ascending

unemployment_df.sort_values(by="Date", ascending=True, inplace=True)

unemployment_df.reset_index(drop=True, inplace=True)

unemployment_df.head()

,Date,Year,month,"Unemployment Rate, 16yo and over","Unemployment Rate, Men, 16yo and over","Unemployment Rate, Women, 16yo and over","Unemployment Rate, White, 16yo and over","Unemployment Rate, White, Men, 16yo and over","Unemployment Rate, White, Women, 16yo and over","Unemployment Rate, African American, 16yo and over",...,"Unemployment Rate, 55yo and over","Unemployment Rate, Men, 55yo and over","Unemployment Rate, Women, 55yo and over","Unemployment Rate, 25yo and over","Unemployment Rate, Men, 25yo and over","Unemployment Rate, Women, 25yo and over","Unemployment Rate, Less than High School diploma, 25yo and over","Unemployment Rate, High school-no college, 25yo and over","Unemployment Rate, Some college, 25yo and over","Unemployment Rate, Bachelor's degree or higher, 25yo and over"
0,2006-01,2006,January,4.7,4.6,4.8,4.1,4.1,4.2,8.9,...,3.1,3.2,3.1,3.7,3.5,3.9,7.0,4.4,3.6,2.1
1,2006-02,2006,February,4.8,4.8,4.8,4.1,4.2,4.1,9.5,...,2.9,2.8,3.0,3.7,3.7,3.9,7.1,4.4,3.6,2.2
2,2006-03,2006,March,4.7,4.7,4.7,4.0,4.0,4.0,9.5,...,2.6,2.6,2.7,3.6,3.5,3.8,6.9,4.1,3.7,2.2
3,2006-04,2006,April,4.7,4.7,4.7,4.1,4.1,4.0,9.4,...,3.0,3.2,3.0,3.7,3.6,3.9,6.9,4.4,3.8,2.2
4,2006-05,2006,May,4.6,4.7,4.5,4.1,4.2,3.9,8.7,...,3.0,3.0,2.9,3.7,3.6,3.7,7.0,4.3,3.8,2.1


In [10]:
# Merge construction (housing indicator) and unemployment rate (by demographic)

construction_unemployment_rate_df = pd.merge(cs_df, unemployment_df, on="Date", how="outer")
construction_unemployment_rate_df.to_csv("construction_unemployment_rate.csv")
construction_unemployment_rate_df

,Date,Total Construction,Total Residential,Total Nonresidential,Total Lodging,Total Office,Total Commercial,Total Health Care,Total Educational,Total Religious,...,"Unemployment Rate, 55yo and over","Unemployment Rate, Men, 55yo and over","Unemployment Rate, Women, 55yo and over","Unemployment Rate, 25yo and over","Unemployment Rate, Men, 25yo and over","Unemployment Rate, Women, 25yo and over","Unemployment Rate, Less than High School diploma, 25yo and over","Unemployment Rate, High school-no college, 25yo and over","Unemployment Rate, Some college, 25yo and over","Unemployment Rate, Bachelor's degree or higher, 25yo and over"
0,2006-01,1195698.0,681319.0,514379.0,13561.0,46915.0,76900.0,35678.0,83383.0,7435.0,...,3.1,3.2,3.1,3.7,3.5,3.9,7.0,4.4,3.6,2.1
1,2006-02,1205060.0,683903.0,521157.0,14272.0,49018.0,74345.0,36434.0,83987.0,7839.0,...,2.9,2.8,3.0,3.7,3.7,3.9,7.1,4.4,3.6,2.2
2,2006-03,1205018.0,674306.0,530712.0,15619.0,49179.0,74922.0,36751.0,84429.0,8032.0,...,2.6,2.6,2.7,3.6,3.5,3.8,6.9,4.1,3.7,2.2
3,2006-04,1192849.0,656452.0,536398.0,17444.0,52392.0,73856.0,37058.0,83236.0,7817.0,...,3.0,3.2,3.0,3.7,3.6,3.9,6.9,4.4,3.8,2.2
4,2006-05,1180699.0,640166.0,540532.0,17269.0,53233.0,75490.0,37688.0,83137.0,7911.0,...,3.0,3.0,2.9,3.7,3.6,3.7,7.0,4.3,3.8,2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2015-08,1162172.0,434810.0,727361.0,24146.0,57531.0,68118.0,40361.0,89424.0,3707.0,...,3.8,3.9,3.6,4.1,4.0,4.2,7.9,5.5,4.3,2.4
116,2015-09,1158906.0,441998.0,716907.0,23800.0,56592.0,68225.0,39576.0,87112.0,3878.0,...,3.8,4.0,3.7,4.1,4.1,4.2,7.9,5.3,4.3,2.5
117,2015-10,1147465.0,443856.0,703610.0,23208.0,56247.0,69460.0,39498.0,84002.0,3750.0,...,3.5,3.4,3.5,4.1,4.1,4.2,7.6,5.2,4.4,2.5
118,2015-11,1147122.0,444841.0,702281.0,22797.0,58025.0,69055.0,39798.0,85696.0,3481.0,...,3.7,3.7,3.7,4.2,4.1,4.3,6.8,5.4,4.4,2.5


## Statewide unemployment rate

In [11]:
# Pull state data from Bureau of Labor Statistics

url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers = {"content-type": "application/json"}

target_series = ["LASST010000000000003", "LASST020000000000003", "LASST040000000000003", "LASST050000000000003",
                 "LASST060000000000003", "LASST080000000000003", "LASST090000000000003", "LASST100000000000003",
                 "LASST110000000000003", "LASST120000000000003", "LASST130000000000003", "LASST150000000000003",
                 "LASST160000000000003", "LASST170000000000003", "LASST180000000000003", "LASST190000000000003",
                 "LASST200000000000003", "LASST210000000000003", "LASST220000000000003", "LASST230000000000003",
                 "LASST240000000000003", "LASST250000000000003", "LASST260000000000003", "LASST270000000000003",
                 "LASST280000000000003", "LASST290000000000003", "LASST300000000000003", "LASST310000000000003",
                 "LASST320000000000003", "LASST330000000000003", "LASST340000000000003", "LASST350000000000003",
                 "LASST360000000000003", "LASST370000000000003", "LASST380000000000003", "LASST390000000000003",
                 "LASST400000000000003", "LASST410000000000003", "LASST420000000000003", "LASST720000000000003",
                 "LASST440000000000003", "LASST450000000000003", "LASST460000000000003", "LASST470000000000003",
                 "LASST480000000000003", "LASST490000000000003", "LASST500000000000003", "LASST510000000000003",
                 "LASST530000000000003", "LASST540000000000003"]




parameters = json.dumps({"seriesid": target_series,
             "startyear": start_year,
             "endyear": end_year,
             "registrationkey": bls_api_key})

p = requests.post(url, data=parameters, headers=headers)

json_data2 = json.loads(p.text)

In [12]:
# Pull state data from Bureau of Labor Statistics

url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers = {"content-type": "application/json"}

target_series = ["LASST550000000000003", "LASST560000000000003"]




parameters = json.dumps({"seriesid": target_series,
             "startyear": start_year,
             "endyear": end_year,
             "registrationkey": bls_api_key})

p = requests.post(url, data=parameters, headers=headers)

json_data3 = json.loads(p.text)

In [33]:
Year = []
month = []
date = []

for data_point in np.arange(len(json_data2["Results"]["series"][0]["data"])):
        Year.append(f"{json_data2['Results']['series'][0]['data'][data_point]['year']}")
        month.append(f"{json_data2['Results']['series'][0]['data'][data_point]['periodName']}")
        date.append(f"{json_data2['Results']['series'][0]['data'][data_point]['year']}-{json_data2['Results']['series'][0]['data'][data_point]['period'][1:]}")

In [34]:
# Create DataFrame

state_unemployment_df = pd.DataFrame({"Date": date, "Year": Year, "month": month,
                                "LASST010000000000003": "", "LASST020000000000003": "", "LASST040000000000003": "",
                                "LASST050000000000003": "", "LASST060000000000003": "", "LASST080000000000003": "",
                                "LASST090000000000003": "", "LASST100000000000003": "", "LASST110000000000003": "",
                                "LASST120000000000003": "", "LASST130000000000003": "", "LASST150000000000003": "",
                                "LASST160000000000003": "", "LASST170000000000003": "", "LASST180000000000003": "",
                                "LASST190000000000003": "", "LASST200000000000003": "", "LASST210000000000003": "",
                                "LASST220000000000003": "", "LASST230000000000003": "", "LASST240000000000003": "",
                                "LASST250000000000003": "", "LASST260000000000003": "", "LASST270000000000003": "",
                                "LASST280000000000003": "", "LASST290000000000003": "", "LASST300000000000003": "",
                                "LASST310000000000003": "", "LASST320000000000003": "", "LASST330000000000003": "",
                                "LASST340000000000003": "", "LASST350000000000003": "", "LASST360000000000003": "",
                                "LASST370000000000003": "", "LASST380000000000003": "", "LASST390000000000003": "",
                                "LASST400000000000003": "", "LASST410000000000003": "", "LASST420000000000003": "",
                                "LASST720000000000003": "", "LASST440000000000003": "", "LASST450000000000003": "",
                                "LASST460000000000003": "", "LASST470000000000003": "", "LASST480000000000003": "",
                                "LASST490000000000003": "", "LASST500000000000003": "", "LASST510000000000003": "",
                                "LASST530000000000003": "", "LASST540000000000003": "", "LASST550000000000003": "",
                                "LASST560000000000003": ""})
state_unemployment_df.head()

,Date,Year,month,LASST010000000000003,LASST020000000000003,LASST040000000000003,LASST050000000000003,LASST060000000000003,LASST080000000000003,LASST090000000000003,...,LASST460000000000003,LASST470000000000003,LASST480000000000003,LASST490000000000003,LASST500000000000003,LASST510000000000003,LASST530000000000003,LASST540000000000003,LASST550000000000003,LASST560000000000003
0,2015-12,2015,December,,,,,,,,...,,,,,,,,,,
1,2015-11,2015,November,,,,,,,,...,,,,,,,,,,
2,2015-10,2015,October,,,,,,,,...,,,,,,,,,,
3,2015-09,2015,September,,,,,,,,...,,,,,,,,,,
4,2015-08,2015,August,,,,,,,,...,,,,,,,,,,


In [35]:
# Fills in values

for series2 in np.arange(len(json_data2["Results"]["series"])):
    for data_point2 in np.arange(len(json_data2["Results"]["series"][series2]["data"])):
        state_unemployment_df.loc[data_point2, json_data2["Results"]["series"][series2]["seriesID"]] = json_data2["Results"]["series"][series2]["data"][data_point2]["value"]

for series3 in np.arange(len(json_data3["Results"]["series"])):
    for data_point3 in np.arange(len(json_data3["Results"]["series"][series3]["data"])):
        state_unemployment_df.loc[data_point3, json_data3["Results"]["series"][series3]["seriesID"]] = json_data3["Results"]["series"][series3]["data"][data_point3]["value"]

In [36]:
# Sets data to floats

for column in state_unemployment_df.columns[3:]:
    state_unemployment_df[f"{column}"] = state_unemployment_df[f"{column}"].astype(float)

# Rename Column labels
    
state_unemployment_df.rename(columns={"LASST010000000000003" : "Alabama", "LASST020000000000003" : "Alaska",
                                      "LASST040000000000003" : "Arizona", "LASST050000000000003" : "Arkansas",
                                      "LASST060000000000003" : "California", "LASST080000000000003" : "Colorado",
                                      "LASST090000000000003" : "Connecticut", "LASST100000000000003" : "Delaware",
                                      "LASST110000000000003" : "District Of Columbia", "LASST120000000000003" : "Florida",
                                      "LASST130000000000003" : "Georgia", "LASST150000000000003" : "Hawaii",
                                      "LASST160000000000003" : "Idaho", "LASST170000000000003" : "Illinois",
                                      "LASST180000000000003" : "Indiana", "LASST190000000000003" : "Iowa",
                                      "LASST200000000000003" : "Kansas", "LASST210000000000003" : "Kentucky",
                                      "LASST220000000000003" : "Louisiana", "LASST230000000000003" : "Maine",
                                      "LASST240000000000003" : "Maryland", "LASST250000000000003" : "Massachusetts",
                                      "LASST260000000000003" : "Michigan", "LASST270000000000003" : "Minnesota",
                                      "LASST280000000000003" : "Mississippi", "LASST290000000000003" : "Missouri",
                                      "LASST300000000000003" : "Montana", "LASST310000000000003" : "Nebraska",
                                      "LASST320000000000003" : "Nevada", "LASST330000000000003" : "New Hampshire",
                                      "LASST340000000000003" : "New Jersey", "LASST350000000000003" : "New Mexico",
                                      "LASST360000000000003" : "New York", "LASST370000000000003" : "North Carolina",
                                      "LASST380000000000003" : "North Dakota", "LASST390000000000003" : "Ohio",
                                      "LASST400000000000003" : "Oklahoma", "LASST410000000000003" : "Oregon",
                                      "LASST420000000000003" : "Pennsylvania", "LASST720000000000003" : "Puerto Rico",
                                      "LASST440000000000003" : "Rhode Island", "LASST450000000000003" : "South Carolina",
                                      "LASST460000000000003" : "South Dakota", "LASST470000000000003" : "Tennessee",
                                      "LASST480000000000003" : "Texas", "LASST490000000000003" : "Utah",
                                      "LASST500000000000003" : "Vermont", "LASST510000000000003" : "Virginia",
                                      "LASST530000000000003" : "Washington", "LASST540000000000003" : "West Virginia",
                                      "LASST550000000000003" : "Wisconsin", "LASST560000000000003" : "Wyoming"},
                             inplace=True)

#Sort data ascending

state_unemployment_df.sort_values(by="Date", ascending=True, inplace=True)

state_unemployment_df.reset_index(drop=True, inplace=True)

state_unemployment_df.to_csv("State_Unemployment_Rates.csv", index=False)

state_unemployment_df

,Date,Year,month,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,2006-01,2006,January,4.1,6.9,4.5,5.0,5.0,4.6,4.5,...,3.2,5.3,5.2,3.5,3.5,3.2,5.1,4.7,4.7,3.4
1,2006-02,2006,February,4.1,6.9,4.4,5.0,5.0,4.6,4.4,...,3.1,5.3,5.1,3.4,3.5,3.1,5.0,4.7,4.7,3.3
2,2006-03,2006,March,4.1,6.8,4.4,5.1,4.9,4.5,4.4,...,3.0,5.3,5.1,3.3,3.5,3.1,5.0,4.7,4.7,3.2
3,2006-04,2006,April,4.1,6.7,4.4,5.1,4.9,4.5,4.3,...,3.0,5.4,5.1,3.2,3.5,3.1,5.0,4.8,4.7,3.1
4,2006-05,2006,May,4.1,6.7,4.3,5.2,4.9,4.4,4.3,...,3.0,5.5,5.1,3.1,3.6,3.1,5.1,5.0,4.7,3.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2015-08,2015,August,6.1,6.5,6.0,4.8,6.0,3.7,5.6,...,3.1,5.5,4.4,3.6,3.5,4.3,5.6,6.7,4.4,4.4
116,2015-09,2015,September,6.1,6.5,5.9,4.7,5.9,3.6,5.6,...,3.0,5.3,4.4,3.6,3.5,4.2,5.6,6.6,4.4,4.4
117,2015-10,2015,October,6.0,6.6,5.9,4.5,5.8,3.6,5.6,...,2.9,5.2,4.5,3.6,3.4,4.1,5.6,6.5,4.3,4.5
118,2015-11,2015,November,6.0,6.6,5.8,4.4,5.8,3.5,5.6,...,2.9,5.0,4.5,3.6,3.4,4.1,5.6,6.4,4.3,4.6


In [38]:
state_unemployment_df.drop(["Date", "month"], axis=1, inplace=True)
state_unemployment_df = state_unemployment_df.groupby("Year").mean()
state_unemployment_df = state_unemployment_df.transpose()
states_df = pd.read_csv("Resources/states.tsv", delimiter="\t")
state_unemployment_df = state_unemployment_df.rename_axis("name")
state_unemployment_df = pd.merge(state_unemployment_df, states_df, on="name", how="inner")
state_unemployment_df = state_unemployment_df.drop(["state"], axis=1)
state_unemployment_df.to_csv("state_unempoyment_rate.csv")

## Unemployment level

In [52]:
# Pull unemployment level data from Bureau of Labor Statistics

url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers = {"content-type": "application/json"}

target_series = ["LNS13000000", "LNS13023621", "LNS13023653", "LNS13025699", "LNS13023705", "LNS13023557", "LNS13023569"]




parameters = json.dumps({"seriesid": target_series,
             "startyear": start_year,
             "endyear": end_year,
             "registrationkey": bls_api_key})

p = requests.post(url, data=parameters, headers=headers)

json_data4 = json.loads(p.text)

In [53]:
Year = []
month = []
date = []

for data_point in np.arange(len(json_data4["Results"]["series"][0]["data"])):
        Year.append(f"{json_data4['Results']['series'][0]['data'][data_point]['year']}")
        month.append(f"{json_data4['Results']['series'][0]['data'][data_point]['periodName']}")
        date.append(f"{json_data4['Results']['series'][0]['data'][data_point]['year']}-{json_data4['Results']['series'][0]['data'][data_point]['period'][1:]}")

In [54]:
# Create DataFrame

layoffs_df = pd.DataFrame({"Date": date,
                           "Year": Year,
                           "month": month,
                           "LNS13000000": "",
                           "LNS13023621": "",
                           "LNS13023653": "",
                           "LNS13025699": "",
                           "LNS13023705": "",
                           "LNS13023557": "",
                           "LNS13023569": ""})
layoffs_df.head()

,Date,Year,month,LNS13000000,LNS13023621,LNS13023653,LNS13025699,LNS13023705,LNS13023557,LNS13023569
0,2015-12,2015,December,,,,,,,
1,2015-11,2015,November,,,,,,,
2,2015-10,2015,October,,,,,,,
3,2015-09,2015,September,,,,,,,
4,2015-08,2015,August,,,,,,,


In [55]:
# Fills in values

for series4 in np.arange(len(json_data4["Results"]["series"])):
    for data_point4 in np.arange(len(json_data4["Results"]["series"][series4]["data"])):
        layoffs_df.loc[data_point4, json_data4["Results"]["series"][series4]["seriesID"]] = json_data4["Results"]["series"][series4]["data"][data_point4]["value"]


In [56]:
# Sets data to float

for column in layoffs_df.columns[3:]:
    layoffs_df[f"{column}"] = layoffs_df[f"{column}"].astype(float)

# Rename column labels

layoffs_df.rename(columns={"LNS13000000": "Unemployment level, 16yo and over, thousands",
                           "LNS13023621": "Job losers, 16yo and over, thousands",
                           "LNS13023653": "Job losers on layoff, 16yo and over, thousands",
                           "LNS13025699": "Job losers not on layoff, 16yo and over, thousands",
                           "LNS13023705": "Job leavers (quit), 16yo and over, thousands",
                           "LNS13023557": "Reentrants to labor force, 16yo and over, thousands",
                           "LNS13023569": "New entrants to labor force, 16yo and over, thousands"},
                 inplace=True)

# Sort data ascending

layoffs_df.sort_values(by="Date", ascending=True, inplace=True)

layoffs_df.reset_index(drop=True, inplace=True)

layoffs_df.to_csv("Unemployment_Levels.csv", index=False)

layoffs_df

,Date,Year,month,"Unemployment level, 16yo and over, thousands","Job losers, 16yo and over, thousands","Job losers on layoff, 16yo and over, thousands","Job losers not on layoff, 16yo and over, thousands","Job leavers (quit), 16yo and over, thousands","Reentrants to labor force, 16yo and over, thousands","New entrants to labor force, 16yo and over, thousands"
0,2006-01,2006,January,7064.0,3349.0,864.0,2485.0,821.0,2261.0,616.0
1,2006-02,2006,February,7184.0,3366.0,890.0,2477.0,872.0,2316.0,711.0
2,2006-03,2006,March,7072.0,3399.0,912.0,2486.0,833.0,2174.0,636.0
3,2006-04,2006,April,7120.0,3505.0,886.0,2619.0,853.0,2172.0,591.0
4,2006-05,2006,May,6980.0,3473.0,930.0,2543.0,862.0,2135.0,517.0
...,...,...,...,...,...,...,...,...,...,...
115,2015-08,2015,August,7992.0,3973.0,944.0,3029.0,775.0,2372.0,842.0
116,2015-09,2015,September,7907.0,3877.0,917.0,2960.0,779.0,2427.0,840.0
117,2015-10,2015,October,7922.0,3995.0,997.0,2998.0,785.0,2413.0,825.0
118,2015-11,2015,November,8000.0,3903.0,898.0,3005.0,783.0,2443.0,871.0


## COVID cases

In [57]:
url = "https://covidtracking.com/api/us/daily"

response = requests.get(url).json()

# print(json.dumps(response, indent=True))

In [58]:
dates = []
cum_positives = []
cum_deaths = []
inc_positives = []
inc_deaths = []

In [59]:
for day in np.arange(len(response)):
    dates.append(response[day]["date"])
    cum_positives.append(response[day]["positive"])
    cum_deaths.append(response[day]["death"])
    inc_positives.append(response[day]["positiveIncrease"])
    inc_deaths.append(response[day]["deathIncrease"])

In [60]:
sick_df = pd.DataFrame({"Date" : dates,
                       "Total Cases" : cum_positives,
                       "Total Deaths" : cum_deaths,
                       "Daily increase in Cases" : inc_positives,
                       "Daily increase in Deaths" : inc_deaths})

sick_df.loc[len(inc_positives)-1, "Daily increase in Deaths"] = sick_df.loc[len(cum_positives)-1, "Total Deaths"]
sick_df.loc[len(inc_deaths)-1, "Daily increase in Cases"] = sick_df.loc[len(cum_deaths)-1, "Total Cases"]

sick_df["Date"] = sick_df["Date"].astype(str)

In [61]:
for x in np.arange(len(dates)):
    sick_df.loc[x,"Date"] = f"{sick_df.loc[x,'Date'][0:4]}-{sick_df.loc[x,'Date'][4:6]}-{sick_df.loc[x,'Date'][6:]}"

sick_df.sort_values(by="Date", ascending=True, inplace=True)

sick_df.reset_index(drop=True, inplace=True)

In [62]:
sick_df.to_csv("COVID_cases.csv", index=False)
sick_df

,Date,Total Cases,Total Deaths,Daily increase in Cases,Daily increase in Deaths
0,2020-02-28,9,4,9.0,4.0
1,2020-02-29,18,5,9.0,1.0
2,2020-03-01,40,8,22.0,3.0
3,2020-03-02,53,11,13.0,3.0
4,2020-03-03,94,14,41.0,3.0
5,2020-03-04,209,16,115.0,2.0
6,2020-03-05,276,20,67.0,4.0
7,2020-03-06,394,26,118.0,6.0
8,2020-03-07,550,27,156.0,1.0
9,2020-03-08,731,31,181.0,4.0


## Economic Indicators

In [63]:
# csv files are downloaded from Yahoo Finance

file_path1 = "Resources/2020_GSPC.csv"
file_path2 = "Resources/2020_DJI.csv"
file_path3 = "Resources/2020_IXIC.csv"
file_path4 = "Resources/2020_RUT.csv"

SP500_df = pd.read_csv(file_path1)
Dow_df = pd.read_csv(file_path2)
NASDAQ_df = pd.read_csv(file_path3)
Russell2000_df = pd.read_csv(file_path4)

In [64]:
# Reduce number of columns

SP500_df = SP500_df[["Date", "Close", "Volume"]]
Dow_df = Dow_df[["Date", "Close", "Volume"]]
NASDAQ_df = NASDAQ_df[["Date", "Close", "Volume"]]
Russell2000_df = Russell2000_df[["Date", "Close", "Volume"]]

In [65]:
# Merge stock data

eco_ind_df = pd.merge(SP500_df, Dow_df, on="Date", how="outer", suffixes=["_SP500", "_Dow_Jones_Industrial_Average"])
eco_ind_2_df = pd.merge(NASDAQ_df, Russell2000_df, on="Date", how="outer", suffixes=["_NASDAQ", "_Russell2000"])
eco_ind_df = pd.merge(eco_ind_df, eco_ind_2_df, on="Date", how="outer")

eco_ind_df

,Date,Close_SP500,Volume_SP500,Close_Dow_Jones_Industrial_Average,Volume_Dow_Jones_Industrial_Average,Close_NASDAQ,Volume_NASDAQ,Close_Russell2000,Volume_Russell2000
0,2019-10-14,2966.149902,2557020000,26787.359375,178620000,8048.649902,1419730000,1505.430054,25570200
1,2019-10-15,2995.679932,3340740000,27024.800781,245510000,8148.709961,1836650000,1523.300049,33407400
2,2019-10-16,2989.689941,3222570000,27001.980469,214660000,8124.180176,1886720000,1525.060059,32225700
3,2019-10-17,2997.949951,3115960000,27025.880859,222540000,8156.850098,1861570000,1541.839966,31159600
4,2019-10-18,2986.199951,3264290000,26770.199219,288970000,8089.540039,2012930000,1535.479980,32642900
...,...,...,...,...,...,...,...,...,...
119,2020-04-03,2488.649902,6087190000,21052.529297,450010000,7373.080078,3279100000,1052.050049,60871900
120,2020-04-06,2663.679932,6391860000,22679.990234,610760000,7913.240234,3849100000,1138.780029,63918600
121,2020-04-07,2659.409912,7040720000,22653.859375,594660000,7887.259766,4069410000,1139.170044,70407200
122,2020-04-08,2749.979980,5856370000,23433.570313,472740000,8090.899902,3470730000,1191.660034,58563700


In [66]:
COVID_econ_df = pd.merge(sick_df, eco_ind_df, on="Date", how="inner")
COVID_econ_df = COVID_econ_df.drop(["Daily increase in Cases", "Daily increase in Deaths"], axis=1)
COVID_econ_df.to_csv("COVID_econ.csv")
COVID_econ_df

,Date,Total Cases,Total Deaths,Close_SP500,Volume_SP500,Close_Dow_Jones_Industrial_Average,Volume_Dow_Jones_Industrial_Average,Close_NASDAQ,Volume_NASDAQ,Close_Russell2000,Volume_Russell2000
0,2020-02-28,9,4,2954.219971,8563850000,25409.359375,915990000,8567.370117,5301170000,1476.430054,85638500
1,2020-03-02,53,11,3090.229980,6376400000,26703.320313,637200000,8952.169922,4232760000,1518.489990,63764000
2,2020-03-03,94,14,3003.370117,6355940000,25917.410156,647080000,8684.089844,4336700000,1486.079956,63559400
3,2020-03-04,209,16,3130.120117,5035480000,27090.859375,457590000,9018.089844,3602870000,1531.199951,50354800
4,2020-03-05,276,20,3023.939941,5575550000,26121.279297,477370000,8738.589844,3748090000,1478.819946,55755500
5,2020-03-06,394,26,2972.370117,6552140000,25864.779297,599780000,8575.620117,4279850000,1449.219971,65521400
6,2020-03-09,1048,35,2746.560059,8423050000,23851.019531,750430000,7950.680176,4530350000,1313.439941,84230500
7,2020-03-10,1365,37,2882.229980,7635960000,25018.160156,654860000,8344.250000,4431930000,1350.900024,76359600
8,2020-03-11,1757,43,2741.379883,7374110000,23553.220703,663960000,7952.049805,4273890000,1264.300049,73741100
9,2020-03-12,2221,51,2480.639893,8829380000,21200.619141,908260000,7201.799805,5066530000,1122.930054,88293800


In [67]:
# Downloaded csv files from Yahoo Finance

file_path5 = "Resources/2007_GSPC.csv"
file_path6 = "Resources/2007_DJI.csv"
file_path7 = "Resources/2007_IXIC.csv"
file_path8 = "Resources/2007_RUT.csv"

SP500_2_df = pd.read_csv(file_path5)
Dow_2_df = pd.read_csv(file_path6)
NASDAQ_2_df = pd.read_csv(file_path7)
Russell2000_2_df = pd.read_csv(file_path8)

In [68]:
# Reduce the number of columns

SP500_2_df = SP500_2_df[["Date", "Close", "Volume"]]
Dow_2_df = Dow_2_df[["Date", "Close", "Volume"]]
NASDAQ_2_df = NASDAQ_2_df[["Date", "Close", "Volume"]]
Russell2000_2_df = Russell2000_2_df[["Date", "Close", "Volume"]]

In [69]:
# Merge stock data

eco_ind_3_df = pd.merge(SP500_2_df, Dow_2_df, on="Date", how="outer", suffixes=["_SP500", "_Dow_Jones_Industrial_Average"])
eco_ind_4_df = pd.merge(NASDAQ_2_df, Russell2000_2_df, on="Date", how="outer", suffixes=["_NASDAQ", "_Russell2000"])
eco_ind_3_df = pd.merge(eco_ind_3_df, eco_ind_4_df, on="Date", how="outer")

# Set date format

eco_ind_3_df["Date"] = eco_ind_3_df["Date"].str[:7]

eco_ind_3_df

,Date,Close_SP500,Volume_SP500,Close_Dow_Jones_Industrial_Average,Volume_Dow_Jones_Industrial_Average,Close_NASDAQ,Volume_NASDAQ,Close_Russell2000,Volume_Russell2000
0,2006-01,1280.079956,49211650000,10864.860352,6552210000,2305.820068,41781780000,733.200012,492116500
1,2006-02,1280.660034,42859940000,10993.410156,5759090000,2281.389893,37103890000,730.640015,428599400
2,2006-03,1294.869995,50905040000,11109.320313,6680180000,2339.790039,47567350000,765.140015,509050400
3,2006-04,1310.609985,43308430000,11367.139648,5584020000,2322.570068,39227480000,764.539978,433084300
4,2006-05,1270.089966,54312830000,11168.309570,7360590000,2178.879883,45923780000,721.010010,543128300
...,...,...,...,...,...,...,...,...,...
115,2015-08,1972.180054,84626790000,16528.029297,2726340000,4812.709961,45767180000,1159.449951,845967900
116,2015-09,1920.030029,79989370000,16284.700195,2724080000,4620.160156,41503920000,1100.689941,799583700
117,2015-10,2079.360107,85844900000,17663.539063,2605470000,5053.750000,42348240000,1161.859985,858049000
118,2015-11,2080.409912,75943590000,17719.919922,2334610000,5108.669922,36834860000,1198.109985,759135900
